# Function calling

Function calling is a powerful feature of OpenAI `gpt-4` and other models. It essentially is a **Function Picker and Parameter Provider**. It can help you choose which function, if any, to invoke with which parameters, under provided context and instruction. 

LionAGI allows simple usage of function callings in the `Session` object. 

In [1]:
import lionagi as li

In [2]:
from lionagi.schema import BaseTool
# let us define a function description in OpenAI schema

tool_1=[
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Perform multiplication on two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "number1": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 5.34",
                    },
                    "number2": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 17",
                    },
                },
                # specify which parameters are required for the model to respond when function calling
                "required": ["number1", "number2"],
            },
        }
    }
]

def multiply(number1, number2):
    return number1*number2

tools = BaseTool(func=multiply)


In [3]:
tools.func.__name__

'multiply'

In [4]:
# and now some messages
system = "you are asked to perform as a function picker and parameter provider"
task = "Think step by step, understand the following basic math question and provide parameters for function calling."

# when using respond_mode as json to enforce output format, you need to provide specifying details in instruction
json_format = {"number1": "x", "number2": "y"}

instruct1 = {"Task": task, "json_format": json_format}

In [5]:
question = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"
question2 = "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"

context1 = {"Question1": question, "question2": question2}

In [6]:
session = li.Session(system=system)

session.register_tools(tools)
session.llmconfig.update({
    "tools": tool_1, 
    "temperature":0.35,
    "tool_choice": "auto", 
    "response_format": {'type':'json_object'}
})

In [7]:
response = await session.initiate(instruction=instruct1, context=context1)

In [8]:
li.l_call(response, lambda x: print(x));


{
  "tool_uses": [
    {
      "recipient_name": "functions.multiply",
      "parameters": {
        "number1": 3, 	"number2": 4
      }
    },
    {
      "recipient_name": "functions.multiply",
      "parameters": {
        "number1": 3, 	"number2": 2
      }
    }
  ]
}


In [9]:
li.l_call(session.conversation.messages, lambda x: print(f'{x} \n'));

{'role': 'system', 'content': 'you are asked to perform as a function picker and parameter provider'} 

{'role': 'user', 'content': '{"instruction": {"Task": "Think step by step, understand the following basic math question and provide parameters for function calling.", "json_format": {"number1": "x", "number2": "y"}}, "context": {"Question1": "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?", "question2": "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"}}'} 

{'role': 'assistant', 'content': '\n{\n  "tool_uses": [\n    {\n      "recipient_name": "functions.multiply",\n      "parameters": {\n        "number1": 3, \t"number2": 4\n      }\n    },\n    {\n      "recipient_name": "functions.multiply",\n      "parameters": {\n        "number1": 3, \t"number2": 2\n      }\n    }\n  ]\n}'} 

{'role': 'assistant'